In [204]:
# import necessary variables
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

## Load the data

In [205]:
df = pd.read_csv('data/FAOSTAT_data.csv')
df.head()

Domain Code                             Domain  Area Code (M49)  \
0          FS  Suite of Food Security Indicators                4   
1          FS  Suite of Food Security Indicators                4   
2          FS  Suite of Food Security Indicators                4   
3          FS  Suite of Food Security Indicators                4   
4          FS  Suite of Food Security Indicators                4   

          Area  Element Code Element Item Code  \
0  Afghanistan          6121   Value     21010   
1  Afghanistan          6121   Value     21010   
2  Afghanistan          6121   Value     21010   
3  Afghanistan          6121   Value     21010   
4  Afghanistan          6121   Value     21010   

                                                Item  Year Code       Year  \
0  Average dietary energy supply adequacy (percen...   20002002  2000-2002   
1  Average dietary energy supply adequacy (percen...   20012003  2001-2003   
2  Average dietary energy supply adequacy (percen...   20022004  2002-2004   
3  Average dietary energy supply adequacy (percen...   20032005  2003-2005   
4  Average dietary energy supply adequacy (percen...   20042006  2004-2006   

  Unit Value Flag Flag Description Note  
0    %    88    E  Estimated value  NaN  
1    %    89    E  Estimated value  NaN  
2    %    92    E  Estimated value  NaN  
3    %    93    E  Estimated value  NaN  
4    %    94    E  Estimated value  NaN

## Clean and tidy the data

In [206]:
# lowercase the columns and remove the space bar
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')

In [207]:
# check the null values
print(df.isnull().sum())

# # drop the null values
# df = df.dropna()

# drop the columns that are not needed
df = df.drop(['note'], axis=1)

# change the column name
df = df.rename(columns={'area': 'country'})

domain_code              0
domain                   0
area_code_(m49)          0
area                     0
element_code             0
element                  0
item_code                0
item                     0
year_code                0
year                     0
unit                     0
value                 9137
flag                     0
flag_description         0
note                129132
dtype: int64


In [210]:
# filter the data

# per capita food supply variability (kcal/cap/day)
df1 = df[df['item'] == 'Per capita food supply variability (kcal/cap/day)']

# convert the value column to float
df1['value'] = pd.to_numeric(df1['value'], errors='coerce')

# group by country and item
df1 = df1.groupby(['country', 'item'], as_index=False)['value'].mean()

# round the value column
df1['value'] = df1['value'].round(2)

## Data Visualization

Per Capita Food Supply Variability (kcal/cap/day)

### First Plot

In [211]:
# choropleth
fig = px.choropleth(df1, locations = 'country',
                    locationmode='country names',
                    color="value",
                    hover_name = 'country',
                    title = 'Per Capita Food Supply Variability (kcal/cap/day)',
                    color_continuous_scale=px.colors.sequential.Plasma
                    )
fig.update_layout(width=1000, height=800)
fig.show()

In [219]:
# time series data
df2 = df[['country', 'year', 'value']]
# convert the value column to float
df2['value'] = pd.to_numeric(df2['value'], errors='coerce')
# group the data by country and year
df2 = df2.groupby(['country', 'year']).mean().reset_index()
df2 = df2.pivot(index='country', columns='year', values='value')
df2.head()

year                  2000  2000-2002        2001  2001-2003        2002  \
country                                                                    
Afghanistan     223.700588      59.38  224.575882      62.94  270.487368   
Albania         481.722273      52.45  530.579524      52.79  520.983636   
Algeria         553.792500      45.26  640.562857      45.01  577.152500   
American Samoa   61.052500        NaN   61.350000        NaN   61.675000   
Andorra          38.786667        NaN   44.412500        NaN   40.442222   

year            2002-2004        2003  2003-2005        2004  2004-2006  ...  \
country                                                                  ...   
Afghanistan         54.16  271.884737      52.62  244.503810      51.23  ...   
Albania             53.08  539.859091      52.88  560.862727      52.99  ...   
Algeria             44.92  653.721364      44.51  667.308636      44.67  ...   
American Samoa        NaN   62.000000        NaN   50.026000        NaN  ...   
Andorra              0.30   41.285556       0.30   42.087778       0.30  ...   

year                  2017  2017-2019         2018  2018-2020         2019  \
country                                                                      
Afghanistan     359.908235  42.872727   307.894000  42.986364   360.781765   
Albania         928.068947  30.909091  1065.682941  31.566667  1084.625882   
Algeria         774.687143  30.085000   771.723810  29.955000   672.041667   
American Samoa   51.228000        NaN    51.228000        NaN    51.218000   
Andorra          51.605000   0.300000    51.642500   0.300000    51.715000   

year            2019-2021         2020  2020-2022         2021        2022  
country                                                                     
Afghanistan     42.558824   399.280000  35.960000   488.925455  426.945556  
Albania         18.375000  1200.874667  18.150000  2105.637778  631.747143  
Algeria         18.253333   848.272222  18.907692  1380.306364  456.571111  
American Samoa        NaN    51.218000        NaN     2.035000    3.020000  
Andorra          0.300000    57.377143        NaN     2.325000    3.020000  

[5 rows x 44 columns]

In [220]:
absolute_change = df2['2021'] - df2['2000']
relative_change = ((df2['2021'] - df2['2000']) / df2['2000']) * 100

In [221]:
df2 = pd.DataFrame({
    'country': df2.index,
    '2000': df2['2000'],
    '2021': df2['2021'],
    'absolute_change': absolute_change,
    'relative_change': relative_change
})

df2['absolute_change'] = df2['absolute_change'].round(2)
df2['relative_change']=df2['relative_change'].apply(lambda x: int(x))
df2['relative_change']= df2['relative_change'].astype(str) + '%'

In [222]:
df2.drop(columns=['country'], inplace=True)
df2.reset_index(inplace=True)
df2.head()

country        2000         2021  absolute_change relative_change
0     Afghanistan  223.700588   488.925455           265.22            118%
1         Albania  481.722273  2105.637778          1623.92            337%
2         Algeria  553.792500  1380.306364           826.51            149%
3  American Samoa   61.052500     2.035000           -59.02            -96%
4         Andorra   38.786667     2.325000           -36.46            -94%

### Second Plot

In [229]:
fig = px.bar(df2, x='country', y=['2000', '2021'],
             title='Comparison of Values in 2000 and 2021',
             labels={'value': 'Value', 'variable': 'Year'},
             hover_data={'absolute_change': True, 'relative_change': True},
             barmode='group')

fig.show()

Number of children under 5 years of age who are stunted (modeled estimates) (million)

In [232]:
# check the item
value_to_check = 'Number of children under 5 years of age who are stunted (modeled estimates) (million)'
if value_to_check in df['item'].values:
    print("Exists")
else:
    print("Does not exist")

Exists


In [236]:
# Number of children under 5 years of age who are stunted (modeled estimates) (million) 
df3 = df[df['item'] == 'Number of children under 5 years of age who are stunted (modeled estimates) (million)']
# convert the value column to float
df3['value'] = pd.to_numeric(df3['value'], errors='coerce')

# group by country and item
df3 = df3.groupby(['country', 'item'], as_index=False)['value'].sum()

# round the value column
# df3['value'] = df3['value'].round(2)
df3.head()

country                                               item  value
0  Afghanistan  Number of children under 5 years of age who ar...   54.3
1      Albania  Number of children under 5 years of age who ar...    0.7
2      Algeria  Number of children under 5 years of age who ar...   11.9
3       Angola  Number of children under 5 years of age who ar...   38.8
4    Argentina  Number of children under 5 years of age who ar...    6.9

In [ ]:
# normalize the data

In [237]:
# choropleth
fig = px.choropleth(df3, locations = 'country',
                    locationmode='country names',
                    color="value",
                    hover_name = 'country',
                    title = 'Number of children under 5 years of age who are stunted (modeled estimates) (million)',
                    color_continuous_scale=px.colors.sequential.Plasma
                    )
fig.update_layout(width=1000, height=800)
fig.show()

In [238]:
# import highchart classes
from highcharts_core.chart import Chart
from highcharts_core.global_options.shared_options import SharedOptions
from highcharts_core.options import HighchartsOptions
from highcharts_core.options.plot_options.bar import BarOptions
from highcharts_core.options.series.bar import BarSeries